# to do
- test set has to be put in a goof form
- the promointerval was taken out until I can put it in a good form as well
- how to change the arrays inside an array as individual columns (otherwise I think SVM etc will not work)

In [1]:
import numpy as np
import pandas 
import matplotlib.pyplot as plt
from sklearn import svm
import numpy as np


In [2]:
#read the files
store = pandas.read_csv('store.csv', sep = ',')
train_initial = pandas.read_csv('train_v2.csv',dtype={"StateHoliday": str})
test = pandas.read_csv('test_v2.csv', sep = ",") 

#take out the sales (we don't have this info in the test)
sales_train = train_initial['Sales']
train = train_initial.drop('Sales', axis=1)


#take out days were it's closed - When 0 customers will be identified in test data
# we will automatically put the test sales at 0.
sales_train = pandas.Series.to_frame(sales_train)
train = train[train.Customers != 0]
sales_train = sales_train[sales_train.Sales != 0]

In [3]:
# put the 2 types of data together
df = train.join(store.set_index('Store'), on='Store')
test = test.join(store.set_index('Store'), on='Store')

# extract the date as day/month/year/weak of the year
df['Date']  = pandas.to_datetime(df['Date'], errors='coerce')
df['Year']  = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['WeekOfYear'] = df['Date'].dt.weekofyear

test['Date']  = pandas.to_datetime(test['Date'], errors='coerce')
test['Year']  = test['Date'].dt.year
test['Month'] = test['Date'].dt.month
test['Day'] = test['Date'].dt.day
test['WeekOfYear'] = test['Date'].dt.weekofyear

In [4]:
# combine train and test (in that oder) to make sure all tranformation are the same for 
# test and train (same dummies etc)
number_train = df.shape[0]
number_test = test.shape[0]
frames = [df,test]
result = pandas.concat(frames)

In [5]:
#fill the missing values with -1
result = result.fillna(-1)

In [6]:
# the code in this cell has been inspired by a similar function foud online at :
# http://www.akbarian.org/notes/sale-forecasting-time-series-analysis/

#we will detect wether there is a promotion in this month:
monthAsString = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
                     7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
result['MonthCompare'] = result.Month.map(monthAsString)

result['IsPromo2Month'] = 0

for interval in result.PromoInterval.unique():
        if interval != -1:
            for month in interval.split(','):
                condition = (result.MonthCompare == month) & (result.PromoInterval == interval)
                # If promo started this year, Week of Year must be > Promo2SinceWeek
                cond1 = (condition & (result.Year == result.Promo2SinceYear)
                         & (result.WeekOfYear >= result.Promo2SinceWeek) )
                # Or If promo started previous year, Week of Year doesn't matter
                cond2 = condition & (result.Year > result.Promo2SinceYear)
                final_condition = cond1 | cond2
                result.loc[final_condition, 'IsPromo2Month'] = 1


# detect if there is competition open at the time of the evaluation
result['Competition'] = 0
first_condition = result.Year > result.CompetitionOpenSinceYear
second_condition = ((result.Year == result.CompetitionOpenSinceYear)
          & (result.Month >= result.CompetitionOpenSinceMonth))
fullcond = first_condition | second_condition
result.loc[fullcond, 'Competition'] = 1

In [7]:
#drop data that is not important anymore
result.drop(['MonthCompare','PromoInterval','Date','Store'], axis=1, inplace=True)

In [8]:
result.head()

,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Year,Month,Day,WeekOfYear,IsPromo2Month,Competition
0,5,616,1,1,0,0,c,a,1270.0,9.0,2008.0,0,-1.0,-1.0,2015,1,30,5,0,1
1,5,624,1,1,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,2015,1,30,5,1,1
2,5,678,1,1,0,0,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,2015,1,30,5,1,1
3,5,1632,1,1,0,0,c,c,620.0,9.0,2009.0,0,-1.0,-1.0,2015,1,30,5,0,1
4,5,617,1,1,0,0,a,a,29910.0,4.0,2015.0,0,-1.0,-1.0,2015,1,30,5,0,0


In [9]:
# def days_between(d1, d2): #d1 and d2 must be datetime object
#     d1 = datetime.strptime(d1, "%Y-%m-%d")
#     d2 = datetime.strptime(d2, "%Y-%m-%d")
#     return abs((d2 - d1).days)

In [10]:
# def scalling (category) :
#     result[category] = result[category] - result[category].mean()
#     
# scalling ('Year')
# scalling('Customers')

# result['CompetitionDistance'] = result['CompetitionDistance'].fillna(0)
# 
# result['CompetitionFar'] = 0*result['CompetitionDistance']


        

        

In [11]:
# for i in range (result.shape[0]) :
#     if result['CompetitionExist'].iloc[i] > 0 :
#         result['CompetitionExist'].iloc[i] = 1
#     else :
#         result['CompetitionExist'].iloc[i] = 0
        
    # if result['CompetitionDistance'].iloc[i] > result['CompetitionDistance'].mean() :
    #     result['CompetitionFar'].iloc[i] = 1
    # else :
    #     result['CompetitionFar'].iloc[i] = 0

In [12]:
# replace the caracters by numbers, to be used in decision trees later
mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
result.StateHoliday.replace(mappings, inplace=True)
result.StoreType.replace(mappings, inplace=True)
result.Assortment.replace(mappings, inplace=True)

In [13]:
# By analizing the data some funny behaviour can happen in the beginning or the end of every month
# to adress this issue we add 2 more parameters to identify the beginning and end of month
# finally in general the month of december behaves very differently than the others
# because of christmas.

result['Friday'] = 0
result['EarlyInMonth'] = 0
result['LateInMonth'] = 0
result['December'] = 0
result.loc[result.DayOfWeek == 5,'Friday'] = 1
result.loc[result.Day < 5, 'EarlyInMonth'] = 1
result.loc[result.Day > 25, 'LateInMonth'] = 1
result.loc[result.Month == 12,'December'] = 1

In [14]:
result = pandas.get_dummies(result, columns=['DayOfWeek', 'StateHoliday', 'StoreType', 
                                             'Assortment', 'Month', 'WeekOfYear', 'CompetitionOpenSinceYear', 
                                             'Promo2SinceYear'], dummy_na = True)

In [15]:
df = result.iloc[:number_train,:]
test = result.iloc[number_train:,:]

In [16]:
df = df.astype(np.float32)
test = test.astype(np.float32)

In [17]:
#put the data as a numpy array
data_train = df.values
#sales_train = sales_train_train.values
sales_train = pandas.DataFrame.as_matrix(sales_train)

In [18]:
sales_train = sales_train.reshape(len(sales_train),)

In [ ]:
clf = svm.SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.05, gamma='auto',
    kernel='poly', max_iter = -1, shrinking=True, tol=0.005, verbose=False, ).fit(data_train, sales_train)

In [ ]:
prediction_of_sales = clf.predict(test)

In [ ]:
for i in range (number_test) :
    if test['Customers'].iloc[i] == 0 :
        prediction_of_sales[i] = 0

In [ ]:
from math import sqrt
def rmspe(y,y_pred):
    summ = 0
    for i in range(len(y)):
        if y[i] != 0:
            summ += (1-y_pred[i]/y[i])**2
    return sqrt(summ/len(y))

In [ ]:
prediction_of_sales_in_sample = clf.predict(data_train)

In [ ]:
rmspe (sales_train, prediction_of_sales_in_sample)

In [ ]:
submission = pandas.DataFrame()
submission['Sales'] = prediction_of_sales
# submission.Sales = submission.Sales.astype(int)
cols = ['Id','Sales']
submission['Id'] = submission.index + 1
submission = submission[cols]
submission.to_csv('submission_svm.csv', index=False)